In [1]:
# Libraries
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)
import json, snowflake.connector

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# establish the connection to snowflake
ctx = snowflake.connector.connect( 
    **json.load(open('/opt/ich/python-snowflake-defaults.json')))
    
# verify and test if connection is working
try: 
    cs = ctx.cursor() 
    cs.execute('SELECT current_version(), current_role(), current_warehouse()')
    print(cs.fetchone())
finally: 
    cs.close()

In [3]:
# Import 
query = '''
SELECT * from ICHT_SANDBOX_PROD.COVOAM_22016.ANTI_INFECTIVE_ADMINISTRATION
'''
cur = ctx.cursor().execute(query)
drug_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [8]:
# Generate count
anti_infectives_count = drug_df.MEDICATION_NAME_SHORT.value_counts()

In [10]:
# Save
#anti_infectives_count.to_csv('switch_data/anti_infectives_count.csv')

In [5]:
antibiotics = drug_df

In [7]:
# Import
path = r'switch_data/anti_infectives_flag_final.csv'
anti_infectives_flag_final = pd.read_csv(path)

In [9]:
# Filter for relevant delivery methods
antibiotics = antibiotics.groupby('ROUTE').filter(lambda x: len(x) > 3000)
antibiotics['ROUTE'] = antibiotics['ROUTE'].replace({'oral': 'PO', 'enteral':'PO', 'oromucosal':'PO', 'oral/NG':'PO', 'NG': 'PO'})
antibiotics['ADMINISTRATION_DATETIME'] = pd.to_datetime(antibiotics['ADMINISTRATION_DATETIME']).dt.date


In [11]:
# Get list of antibiotics
antibiotic_list = anti_infectives_flag_final[anti_infectives_flag_final['switch_antibiotic_flag'] == 'Yes']['MEDICATION_NAME_SHORT'].to_list()
len(antibiotic_list)

73

In [12]:
# Filter for antibiotics
antibiotics = antibiotics[antibiotics['MEDICATION_NAME_SHORT'].isin(antibiotic_list)]

In [15]:
# Filter columns
antibiotics = antibiotics[['SUBJECT', 'ADMINISTRATION_DATETIME', 'ROUTE']]

In [16]:
antibiotics.SUBJECT.nunique()

53891

In [17]:
antibiotics.ROUTE.value_counts()

IV    547380
PO    305973
Name: ROUTE, dtype: int64

In [18]:
# Filter for those with IV and PO
filtered_antibiotics = antibiotics[antibiotics['SUBJECT'].isin((antibiotics.groupby(['SUBJECT'])['ROUTE'].nunique() > 1).loc[lambda x : x == True].index.to_list())]

In [19]:
filtered_antibiotics.SUBJECT.nunique()

18381

In [ ]:
# Orientate by stay so not so disjointed

In [21]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.EPISODES
'''
cur = ctx.cursor().execute(query)
episodes = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [22]:
# Merge with episodes
antibiotic_episodes = pd.merge(filtered_antibiotics, episodes[['SUBJECT', 'SPELL_IDENTIFIER', 'ADMISSION_DATE_TIME', 'DISCHARGE_DATE_TIME']])

In [23]:
antibiotic_episodes['ADMISSION_DATE_TIME'] = pd.to_datetime(antibiotic_episodes['ADMISSION_DATE_TIME']).dt.date
antibiotic_episodes['DISCHARGE_DATE_TIME'] = pd.to_datetime(antibiotic_episodes['DISCHARGE_DATE_TIME']).dt.date

In [24]:
# Filter for where antibiotic date in spell date range 
antibiotic_episodes = antibiotic_episodes[(antibiotic_episodes['ADMINISTRATION_DATETIME'] >= antibiotic_episodes['ADMISSION_DATE_TIME']) & (antibiotic_episodes['ADMINISTRATION_DATETIME'] <= antibiotic_episodes['DISCHARGE_DATE_TIME'])]
antibiotic_episodes.drop_duplicates(inplace=True)

In [26]:
# Filter for those with IV and PO in same spell
filtered_antibiotic_episodes = antibiotic_episodes[antibiotic_episodes['SPELL_IDENTIFIER'].isin((antibiotic_episodes.groupby(['SUBJECT', 'SPELL_IDENTIFIER'])['ROUTE'].nunique() > 1).loc[lambda x : x == True].index.get_level_values(1).to_list())]

In [ ]:
# Order
filtered_antibiotic_episodes.sort_values(by=['SUBJECT', 'SPELL_IDENTIFIER', 'ADMISSION_DATE_TIME', 'ADMINISTRATION_DATETIME', 'ROUTE'], inplace=True)

In [ ]:
# Create po_flag
def po_flag_fun(row):
    if row['ROUTE'] == 'IV':
        return 0
    elif row['ROUTE'] == 'PO':
        return 1

filtered_antibiotic_episodes['po_flag'] =  filtered_antibiotic_episodes.apply (lambda row: po_flag_fun(row), axis=1)

filtered_antibiotic_episodes.drop_duplicates(subset=['SUBJECT', 'ADMINISTRATION_DATETIME', 'SPELL_IDENTIFIER', 'ADMISSION_DATE_TIME', 'DISCHARGE_DATE_TIME'], inplace=True)

In [ ]:
# Check nonthing weird got thorugh
filtered_antibiotic_episodes[(filtered_antibiotic_episodes['po_flag'] == 1) & (filtered_antibiotic_episodes['ROUTE'] == 'IV')]

In [31]:
def drop_po_first_fun(data):
    bad_codes = []
    for stay_id, sub_df in data.groupby('SPELL_IDENTIFIER'):
        #print(stay_id)
        sub_df.reset_index(inplace=True, drop=True)
        for x in range(len(sub_df)):
            if sub_df.iloc[x]['po_flag'] == 1:
                #print(sub_df.iloc[x])
                bad_codes.append(sub_df.iloc[x]['SPELL_IDENTIFIER'])
            else:
                break
    new_data = data[~data['SPELL_IDENTIFIER'].isin(bad_codes)]
    new_data.reset_index(inplace= True, drop=True)
    return new_data

In [32]:
# Remove those who have oral first
filtered_antibiotic_episodes_2 = drop_po_first_fun(filtered_antibiotic_episodes)

In [33]:
filtered_antibiotic_episodes_2.SPELL_IDENTIFIER.nunique()
filtered_antibiotic_episodes_2.SUBJECT.nunique()

18178

15274

In [34]:
# Filter for those with IV and PO in same spell
filtered_antibiotic_episodes_2 = filtered_antibiotic_episodes_2[filtered_antibiotic_episodes_2['SPELL_IDENTIFIER'].isin((filtered_antibiotic_episodes_2.groupby('SPELL_IDENTIFIER')['po_flag'].nunique() > 1).loc[lambda x : x == True].index.to_list())]

In [35]:
filtered_antibiotic_episodes_2.SPELL_IDENTIFIER.nunique()
filtered_antibiotic_episodes_2.SUBJECT.nunique()

13058

11307

In [36]:
# Change iv_treatment length to prior days treatment length 

# For only having one positive switch day per stay
def iv_treatment_length_fun(data):
    cumcount = []
    count = 0
    pos = -1
    flag = 0

    for x in range(len(data)):
        pos += 1
        if pos == len(data) - 1:
            cumcount.append(count) # add count to last one
            break # end
        elif pos == 0:
            cumcount.append(count) # add 0 to first one
            count += 1
        elif data.iloc[x]['SPELL_IDENTIFIER'] == data.iloc[x+1]['SPELL_IDENTIFIER']:
            if data.iloc[x]['ROUTE'] == 'IV':
                cumcount.append(count)
                count += 1
                flag = 0
            elif flag == 1:
                cumcount.append(999)
                count = 0
                flag = 1
            elif data.iloc[x]['SPELL_IDENTIFIER'] != data.iloc[x-1]['SPELL_IDENTIFIER']:
                cumcount.append(999)
                count = 0
                print('hi')
                print(x)
            else:
                cumcount.append(count)
                count = 0
                flag = 1
        else:
            if data.iloc[x]['ROUTE'] == 'IV':
                cumcount.append(count)
                count = 0
            elif flag == 1:
                cumcount.append(999)
                count = 0
                flag = 0
            else:
                cumcount.append(count)
                count = 0
                flag = 0

    print(len(cumcount))

    data['iv_treatment_length'] = cumcount
    
    return data

In [37]:
filtered_antibiotic_episodes_2 = iv_treatment_length_fun(filtered_antibiotic_episodes_2)

114045


In [38]:
# Re order columns
filtered_antibiotic_episodes_2 = filtered_antibiotic_episodes_2[['SUBJECT', 'SPELL_IDENTIFIER', 'ADMISSION_DATE_TIME', 'DISCHARGE_DATE_TIME', 'ADMINISTRATION_DATETIME', 'ROUTE', 'po_flag', 'iv_treatment_length']]

In [41]:
# Set max iv treatment length
long_stay_list = filtered_antibiotic_episodes_2[(filtered_antibiotic_episodes_2['iv_treatment_length'] > 8) & (filtered_antibiotic_episodes_2['iv_treatment_length'] < 999)]['SPELL_IDENTIFIER'].unique().tolist()

In [42]:
len(long_stay_list)

1495

In [43]:
# Remove those with iv longer than 7 days
filtered_antibiotic_episodes_3 = filtered_antibiotic_episodes_2[~filtered_antibiotic_episodes_2['SPELL_IDENTIFIER'].isin(long_stay_list)]

In [ ]:
# Remove those who have IV after oral

In [45]:
def iv_post_po_fun(data):

    cumcount = []
    flag = 0
    pos = -1

    for x in range(len(data)):
        pos += 1
        if pos == len(data) - 1:
            continue # end
        if data.iloc[x]['SPELL_IDENTIFIER'] == data.iloc[x+1]['SPELL_IDENTIFIER']:
            if data.iloc[x]['ROUTE'] == 'IV':
                if flag == 1:
                    cumcount.append(data.iloc[x]['SPELL_IDENTIFIER'])
                else:
                    continue
            elif data.iloc[x]['ROUTE'] == 'PO':
                flag = 1
                continue
        else:
            if data.iloc[x]['ROUTE'] == 'IV':
                if flag == 1:
                    cumcount.append(data.iloc[x]['SPELL_IDENTIFIER'])
                    flag = 0
                else:
                    continue
                    flag = 0
            elif data.iloc[x]['ROUTE'] == 'PO':
                flag = 0
                continue

    print(len(cumcount))

    data = data[~data['SPELL_IDENTIFIER'].isin(cumcount)]
    
    return data

In [46]:
filtered_antibiotic_episodes_4 = iv_post_po_fun(filtered_antibiotic_episodes_3)

4890


In [49]:
filtered_antibiotic_episodes_4.po_flag.value_counts()
filtered_antibiotic_episodes_4.iv_treatment_length.value_counts()

1    32586
0    29554
Name: po_flag, dtype: int64

999    22276
0      10310
1      10309
2       6894
3       4876
4       3278
5       2043
6       1220
7        662
8        272
Name: iv_treatment_length, dtype: int64

In [50]:
# Save
#filtered_antibiotic_episodes_4.to_csv('switch_data/antibiotic_po_flag.csv', index=False)